# Homework 2


## Mc-Call Job Search Model with Separation

We consider here a very simple job-search model, with separation. Our goal consists in solving this model using a policy iteration algorithm. Some background on the McCall and numerical ideas are discussed on the [QuantEcon](`https://julia.quantecon.org/mccall_model.html`) website.

There is a single worker who can be either employed ("e") or unemployed ("u") in any period.

When unemployed, the jobless worker receives unemployment benefits $c_t=\alpha>0$ in every period as long as he stays unemployed. He also receives a salary offer $w_t$ which is drawn from a discrete i.i.d. distribution and takes values $w_1, ..., w_K$ with probabilities $p_1, ... p_K$ respectively.

When an unemployed worker accepts an offer in period $t$, he gets the salary $w_t$ and becomes employed. He then keeps his salary $w_t$ as long as he stays employed (for $s\geq t$, $c_s=w_t$ if $t$ is the date at which worker got the current job); in each period he has a probability $\lambda$ of becoming unemployed in the next period and remains employed otherwise.

When a worker receives a given amount $x$ his perceived utility is $U(x)=\frac{x^{1-\gamma}}{1-\gamma}$ with $\gamma>1.0$. A worker discounts the future at a rate $\beta \in [0,1[$. As a result, in any period $t_0$ workers seek to maximize $\sum_{t\geq t_0}^{\infty} U(c_t)$.


__Define a parameter type `Parameter`, with fields $\alpha$, $\beta$, $\gamma$, $K$, $\sigma$, $\lambda$. Create a parameter variable $\omega$ with $\alpha=0.5$, $\beta=0.96$, $\gamma=4$, $K=10$, $\sigma=0.6$, $\lambda=0.015$__

In [180]:
struct Parameter
    α::Float64
    β::Float64
    γ::Float64
    K::Float64
    σ::Float64
    λ::Float64
end

ω = Parameter(0.5, 0.96, 4, 10, 0.6, 0.015)

K=10

10

__What is the uniform distribution, whose mean is 1 and standard deviation is $\sigma$? Write a function `discrete_uniform(σ::Float64, K::Int64)::Tuple{Vector{Float64} Vector{Float64}}` to discretize it, using $K$ points. The function should return two vectors `w` and `p` of floats of the same size `K`. Check the results satisfy the right conditions (uniformity, standard deviation).__

Discrete uniform distribution between $a$ and $b$:<br>
Mean is 1:
$$\frac{a+b}{2} = 1 \iff b=2-a$$

Standard deviation is $\sigma$:
$$\frac{(b-a+1)^2 -1}{12} = \sigma^2 \iff \frac{(2-a-a+1)^2 -1}{12}=\sigma^2 \iff \frac{8 + 4a^2 -12a}{12}=\sigma^2 \iff a^2 -3a + 2 - 3\sigma^2 = 0$$
$\Delta = 9  + 12\sigma^2 - 8 = 12 \sigma^2 +1$, hence:
$$x_1 = \frac{3 + \sqrt{1+12\sigma^2}}{2}, x_2 = \frac{3 - \sqrt{1+12\sigma^2}}{2}$$

We check that if $\sigma = 0$, $a = b = 1$:
$$x_1 = \frac{3 + \sqrt{1+12\sigma^2}}{2} =  \frac{3 + \sqrt{1}}{2} = \frac{4}{2} = 2$$
$$x_2 = \frac{3 - \sqrt{1+12\sigma^2}}{2} =  \frac{3 - \sqrt{1}}{2} = \frac{2}{2} = 1$$

So, $a= x_2$, $b = 2 - x_2$.

In [181]:
function discrete_uniform(σ::Float64, K::Int64)
    a = (3 - sqrt(1 + 12*σ^2))/2;
    #print("a", a,"b", 2-a);
    r1 = [a + ((2-2*a)/(K-1))*k for k in 0:K-1];
    r2 = [1/K for k in 0:K-1]
    
    return (r1, r2)
end

discrete_uniform (generic function with 1 method)


The optimal decision of a worker is characterized by two value functions: $V^E(w)$ is the value of being employed at wage $w$ and $V^U(w)$ the value of being unemployed, while receiving job offer $w_t$. In Julia, both $V^U$ and $V^E$ will be represented by arrays `V_U` and `V_E` of size `K`.

A policy `g(w)` is a binary choice in the unemployed state: accept or reject an offer $w_t$. It will then naturally be represented by a boolean array (type `zeros(Bool, K)` to initialize one).

In [182]:
K=5
g = zeros(Bool,K)
typeof(g)

Array{Bool,1}

__Given a policy $g$, write down the recursive equations which defines the corresponding value functions $V^{U,g}(w)$ and $V^{E,g}(w)$.__

In [183]:
function VE(g::Array)
    W,P = discrete_uniform(ω.σ,K)
    return [(W[i]^(1-ω.γ))/(1-ω.γ) for i in 1::K]
    #return [(W[i]^(1-ω.γ))/(1-ω.γ) + β*((1-λ)*VE(g)[i] + λ*sum(VU(g))/K) for i in 1::K]
end

function VU(g)
    W,P = discrete_uniform(ω.σ,K)
    return [g[i]*VE(g)[i] + (1-g[i])*((α^(1-ω.γ))/(1-ω.γ) + β*VU(g)[i]) for i in 1::K]
end

VU (generic function with 1 method)

__Write a function which takes a guess $V^{U,g}(w)$ and $V^{E,g}(w)$ and a policy function $g$ as arguments (and other model parameters) and updates the values, according to the updating equations. This function could have signature `value_update(V_U::Vector{Float64}, V_E::Vector{Float64}, g::Vector{Bool}, ω::Parameter, w::Vector{Float64}, p::Vector{Float64})::Vector{Float64}` where the returned vector has the same size as the supplied ones.__

$${V^E}(w)= U(w)+ \beta((1-\lambda)V^E(w)+\lambda\mathbb{E}[V^U(x)])$$
<br>
$$V^U(w) = \text{max}(V^E(w), U(\alpha) + \beta V^U(w)) = g V^E(w) + (1-g) (U(\alpha) + \beta V^U(w))$$

In [184]:
K=10
                        
function value_update(V_U::Vector{Float64}, V_E::Vector{Float64}, g::Vector{Bool}, ω::Parameter, w::Vector{Float64}, p::Vector{Float64})
    VU_guess = V_U
    VE_guess = V_E
    VU_update =   [g[i]*VE_guess[i] + (1-g[i])*((α^(1-ω.γ))/(1-ω.γ)) + β*VU_guess[i] for i in 1::K]                          
    VE_update =  [(w[i]^(1-ω.γ))/(1-ω.γ) + β*((1-λ)*VE_guess[i] + λ*sum(VU_guess)/K) for i in 1::K]
    return (VU_update, VE_update)
end

value_update (generic function with 1 method)

__Write a function `eval_policy(g::Vector{Bool}, ω::Parameter, w::Vector{Float64}, p::Vector{Float64}, η::Float64)::Tuple{Vector{Float64}, Vector{Float64}}` which iterates on `value_update` find the values that satisfy the evaluation equations for policy `g`.__


In [185]:
VU_guess = [ω.α for i in 1::K ]
VE_guess = [w[i] for i in 1::K]
        
function eval_policy(g::Vector{Bool}, ω::Parameter, w::Vector{Float64}, p::Vector{Float64}, η::Float64)
    VU_update =  [g[i]*VE_guess[i] + (1-g[i])*((α^(1-ω.γ))/(1-ω.γ)) + β*VU_guess[i] for i in 1::K]                               
    VE_update = [(w[i]^(1-ω.γ))/(1-ω.γ) + β*((1-λ)*VE_guess[i] + λ*sum(VU_guess)/K) for i in 1::K]
                            
    while 0.5*(abs(VU_update- VU_guess) + abs(VU_update- VU_guess))>η
        VU_guess = VU_update
        VE_guess = VE_update
        VU_update =  [g[i]*VE_guess[i] + (1-g[i])*((α^(1-ω.γ))/(1-ω.γ)) + β*VU_guess[i] for i in 1::K]                               
        VE_update = [(w[i]^(1-ω.γ))/(1-ω.γ) + β*((1-λ)*VE_guess[i] + λ*sum(VU_guess)/K) for i in 1::K]                                          
    end
    return (VU_update, VE_update)
end


TypeError: TypeError: in typeassert, expected Type, got Int64


__Write a function `improve_policy(g::Vector{Bool}, V_U::Vector{Bool}, V_E::Vector{Bool}, ω::Parameter, w::Vector{Float64}, p::Vector{Float64}, η::Float64)::Vector{Float64}` which returns the improved policy given guesses for the value function(s) at for $t+1$.__

In [188]:
function improve_policy(g::Vector{Bool}, V_U::Vector{Float64}, V_E::Vector{Float64}, ω::Parameter, w::Vector{Float64}, p::Vector{Float64}, η::Float64)
    VU_guess = V_U
    VE_guess = V_E
    
    VU_update, VE_update = eval_policy(g, ω, w, p, η)
    
    g= zeros(Bool,10)
    
    for i in 1::K
        if VE_update[i] > (α^(1-ω.γ))/(1-ω.γ) + β*VU_update[i]
            g[i] = 1
        else
            g[i] = 0
        end
    end
    return g
end

improve_policy (generic function with 2 methods)

__Implement the policy function algorithm. Print the successive approximation errors and comment on the convergence speed.__

In [193]:
function policy_function( V_U::Vector{Float64}, V_E::Vector{Float64}, ω::Parameter, w::Vector{Float64}, p::Vector{Float64}, η::Float64)
    g_guess = zeros(Bool,K)
    g_update = improve_policy(g_guess, V_U, V_E,ω,w,p,η)
    
    g_s = [g_guess, g_update]
    
    while abs(g_guess-g_update)>η
        g_guess = g_update
        g_update = improve_policy(g_guess, V_U, V_E,ω,w,p,η)
        g_s += [g_update]
    end
    return g_s
end

policy_function (generic function with 2 methods)

In [197]:
w, p = discrete_uniform(ω.σ,K)

VU_GUESS = [ω.α,ω.α,ω.α,ω.α,ω.α,ω.α,ω.α,ω.α,ω.α,ω.α]
VE_GUESS = w

print(typeof(VU_GUESS),typeof(VE_GUESS),typeof(ω),typeof(w),typeof(p),typeof(0.1))
policy_function(VU_GUESS, VE_GUESS, ω, w, p, 0.1)


Array{Float64,1}Array{Float64,1}ParameterArray{Float64,1}Array{Float64,1}Float64

TypeError: TypeError: in typeassert, expected Type, got Int64


## Neoclassical growth model (2)

We consider here, another deterministic version of the neoclassical growth model, but propose a slightly different solution method.

A representative agent uses capital $k_t$ to produce $y_t$ using the following production function:

$$y_t = k_t^{\alpha}$$

He chooses to consume an amount $c_t \in ]0, y_t]$ and invests what remains:

$$i_t = y_t - c_t$$.

He accumulates capital $k_t$ according to:

$$k_{t+1} = \left( 1-\delta \right) k_{t} + i_{t}$$

where $\delta$ is the depreciation rate and $i_t$ is the amount invested.

The goal of the representative agent is to maximize:

$$\sum_{t\geq 0} \beta^t U(c_t)$$

where $U(x)=\frac{x^{1-\gamma}}{1-\gamma}$ and $\beta<1$ is the discount factor.

Since the problem is time homogenous, the value function depends on available capital only and satisfies the following Bellman equation:

$$V\left(\underbrace{k}_{k_t}\right) = \max_{c\in[0,1[} U(c) + \beta V\left(\underbrace{(1-\delta)k + \underbrace{(k^{\alpha}-c)}_{y_{t+1}}}_{k_{t+1}}\right)$$

Our goal is to obtain a smooth approximation of $k$ and $V$ by using interpolations techniques.

For this model, using the dynamic first-order conditions, one can show the deterministic steady-state of the model satisfies $1=\beta \left( (1-\delta) + \alpha k^{\alpha -1} \right)$. 

__Create a suitable Parameter type to hold the parameters. Write a function `steady_state(p::Parameter)` to compute the steady-state capital `kbar` and the corresponding steady-state consumption `cbar`__ 


Steady-state capital:
$$1=\beta \left( (1-\delta) + \alpha \bar{k}^{\alpha -1} \right) \iff \bar{k} = (\frac{1 - \beta(1-\delta)}{\beta \alpha})^{\frac{1}{a-1}}$$
Steady-state consumption:
$$(1-\delta)\bar{k} + \bar{k}^a -c = \bar{k} \iff \bar{c} = \bar{k}^a - \delta \bar{k}$$

In [206]:
struct Param
    a::Float64
    δ::Float64
    β::Float64
    γ::Float64
end

p=Param(0.5,0.05,0.96, 4)

Param(0.5, 0.05, 0.96, 4.0)

In [207]:
function steady_state(p::Param)
    kbar = ((1-p.β*(1-p.δ))/(p.β*p.α))^(1/(p.a-1))
    cbar = kbar^p.a - p.δ * kbar
    
    return (kbar,cbar)
end
    

steady_state (generic function with 2 methods)

__Set $N=10$ and define a reasonable grid `kgrid=range(kmin, kmax; length=N)` to approximate capital $k$.__


In [212]:
N=10

#in trillions
kmin = 0.5
kmax = 2

kgrid = range(kmin, kmax; length = N)

0.5:0.16666666666666666:2.0

__The unknown value function is represented as a `N` elements arrays. Define `Vi(k,p)=U(δ * k^α)/(1-𝛽)` and compute the initial guess `V0 = [Vi(k,p) for k in kgrid]`. Define a finer grid `ktest=range(kmin, kmax;length=1000)` and find the values of `Vi` on it by  using `Interpolations.jl` library to interpolate `V0` between the points of `kgrid`.__


In [214]:
using Pkg
Pkg.add("Interpolations")

   Updating registry at `C:\Users\phoeb\.julia\registries\General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  100.0 % %]  22.7 % %Fetching: [==================>                      ]  44.8 %>                  ]  53.0 %Fetching: [==========================>              ]  63.4 %           ]  71.3 %Fetching: [=================================>       ]  80.4 % %]  98.4 %

  Resolving package versions...
  Installed AxisAlgorithms ─── v1.0.0
  Installed Ratios ─────────── v0.4.0
  Installed WoodburyMatrices ─ v0.5.2
  Installed OffsetArrays ───── v1.1.0
  Installed Interpolations ─── v0.12.10
   Updating `C:\Users\phoeb\.julia\environments\v1.4\Project.toml`
  [a98d9a8b] + Interpolations v0.12.10
   Updating `C:\Users\phoeb\.julia\environments\v1.4\Manifest.toml`
  [13072b0f] + AxisAlgorithms v1.0.0
  [a98d9a8b] + Interpolations v0.12.10
  [6fe1bfb0] + OffsetArrays v1.1.0
  [c84ed2f1] + Ratios v0.4.0
  [efce3f68] + WoodburyMatrices v0.5.2


In [216]:
function Vi(k,p)
   return (((p.δ*k^p.a)^(1-p.γ))/(1-p.γ))/(1-p.β) 
end

V0 = [Vi(k,p) for k in kgrid]

ktest=range(kmin, kmax;length=1000)

using Interpolations
itp = = interpolate(ktest, V0, Gridded(Linear()))

┌ Info: Precompiling Interpolations [a98d9a8b-a2ab-59e6-89dd-64a1c18fca59]
└ @ Base loading.jl:1260


LoadError: syntax: unexpected "="


__Compute a Bellman improvement function `bellman(V0::Vector{Float64}, p::Parameter, kgrid)::Tuple{Vector{Float64}, Vector{Float64}}` which does the following steps:__

- take an initial guess `V0` for the value function

- at each grid point from kvec, optimize nonlinearly, the function $c \rightarrow U(c) + \beta V\left((1-\delta)k + (k^{\alpha}-c)\right)$ for each capital level in the grid `kvec`. In this expression the function `V()` interpolates `V0` defined on `kvec` on any point `k` so that the resulting function is continuous. 

- return the updated value and investment rules

In [ ]:
function bellman(V0::Vector{Float64}, p::Param, kgrid)
    
end











__Write a value interation function `vfi(N, p)` which solves the model defined by parameter `p` using the value function algorithm. The function should return the value function and the policy rule.__


__Plot the solution. Comment.__

__Bonus 1: plot a graph showing the convergence back to the steady-state__


__Bonus 2: implement the policy iteration algorithm by adding an evaluation step in the `vfi` function.__
